In [24]:
import numpy as np
import pandas as pd
import os
import time
import itertools

# Imports Machine Learning Classique
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb

#pour acceder au mode gpu
import torch 

#  CONFIGURATION MATÉRIELLE 
# Justification : On vérifie le GPU pour accélérer XGBoost et autres
if not torch.cuda.is_available():
    print("⚠️ Attention : Pas de GPU détecté. Le code sera lent.")
    device_name = "CPU"
else:
    device_name = torch.cuda.get_device_name(0)
    print(f"🚀 Environnement accéléré GPU détecté : {device_name}")
    # On définit le device par défaut pour plus tard (XGBoost en a besoin)
    device = torch.device("cuda")

# CHEMINS
PROJECT_ROOT = r"C:\Users\amisf\Desktop\datascientest_projet"
OUTPUT_DIR = os.path.join(PROJECT_ROOT, "implementation", "outputs")

# LISTE DE RESULTATS (Notre journal de bord)
ALL_RESULTS = []

🚀 Environnement accéléré GPU détecté : NVIDIA GeForce RTX 4070


In [23]:
# CHARGEMENT DATA 
print("📥 Chargement des features ResNet (Numpy)...")
try:
    X_np = np.load(os.path.join(OUTPUT_DIR, 'train_features_resnet50_augmented.npy'))
    y_np = np.load(os.path.join(OUTPUT_DIR, 'train_labels_augmented.npy'))
except:
    print("❌ Erreur : Fichiers .npy introuvables.")
    raise

# Encodage des labels (Transformation en chiffres 0 à 26)
le = LabelEncoder()
y_encoded = le.fit_transform(y_np)
NUM_CLASSES = len(le.classes_)
INPUT_DIM = X_np.shape[1]

# Split Train / Validation (80% / 20%)
print("✂️ Split Train/Val (Numpy)...")
X_train, X_val, y_train, y_val = train_test_split(
    X_np, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# On garde une copie explicite pour les modèles CPU/Sklearn
y_val_cpu = y_val 

print(f"✅ Données prêtes pour le Machine Learning classique.")
print(f"   Train shape : {X_train.shape}")
print(f"   Val shape   : {X_val.shape}")

📥 Chargement des features ResNet (Numpy)...
✂️ Split Train/Val (Numpy)...
✅ Données prêtes pour le Machine Learning classique.
   Train shape : (324000, 2048)
   Val shape   : (81000, 2048)


In [7]:
# GRILLE XGBOOST OPTIMISE + LOGS (Avec barre de progression)
grid_xgb = {
    'n_estimators': [100, 150], 
    'max_depth': [4, 6],
    'learning_rate': [0.1, 0.2],
    'subsample': [0.8]
}

# generation combin
keys, values = zip(*grid_xgb.items())
xgb_configs = [dict(zip(keys, v)) for v in itertools.product(*values)]

print(f"\n🤖 MODE OPTIMISÉ : {len(xgb_configs)} configs a tester...")

for i, conf in enumerate(xgb_configs):
    print(f"\n--- test {i+1}/{len(xgb_configs)} : {conf} ---")
    
    t0 = time.time()
    try:
        # config
        clf = xgb.XGBClassifier(
            objective='multi:softmax',
            num_class=NUM_CLASSES,
            tree_method='hist',
            device='cuda',
            **conf
        )
        
        # ajout eval_set et verbose pour voir la vie
        # verbose=50 affiche une ligne tous les 50 arbres construits
        clf.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            verbose=50 
        )
        
        preds = clf.predict(X_val)
        score = f1_score(y_val_cpu, preds, average='weighted')
        duree = time.time() - t0
        
        print(f"   -> resultat f1: {score:.4f} (duree: {duree:.1f}s)")
        
        ALL_RESULTS.append({
            "Famille": "XGBoost",
            "Details": str(conf),
            "F1-Score": score,
            "Temps": duree
        })
        
    except Exception as e:
        print(f"   erreur config: {e}")


🤖 MODE OPTIMISÉ : 8 configs a tester...

--- test 1/8 : {'n_estimators': 100, 'max_depth': 4, 'learning_rate': 0.1, 'subsample': 0.8} ---
[0]	validation_0-mlogloss:3.08677
[50]	validation_0-mlogloss:1.78692
[99]	validation_0-mlogloss:1.57020


C:\Users\amisf\anaconda3\envs\masterclass_tooling\Lib\site-packages\xgboost\core.py:774: UserWarning: [18:18:56] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\common\error_msg.cc:62: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  return func(**kwargs)


   -> resultat f1: 0.5461 (duree: 142.1s)

--- test 2/8 : {'n_estimators': 100, 'max_depth': 4, 'learning_rate': 0.2, 'subsample': 0.8} ---
[0]	validation_0-mlogloss:2.90037
[50]	validation_0-mlogloss:1.57766
[99]	validation_0-mlogloss:1.38534
   -> resultat f1: 0.5890 (duree: 137.2s)

--- test 3/8 : {'n_estimators': 100, 'max_depth': 6, 'learning_rate': 0.1, 'subsample': 0.8} ---
[0]	validation_0-mlogloss:3.03039
[50]	validation_0-mlogloss:1.53419
[99]	validation_0-mlogloss:1.29716
   -> resultat f1: 0.6271 (duree: 219.3s)

--- test 4/8 : {'n_estimators': 100, 'max_depth': 6, 'learning_rate': 0.2, 'subsample': 0.8} ---
[0]	validation_0-mlogloss:2.79613
[50]	validation_0-mlogloss:1.31443
[99]	validation_0-mlogloss:1.10288
   -> resultat f1: 0.6752 (duree: 213.4s)

--- test 5/8 : {'n_estimators': 150, 'max_depth': 4, 'learning_rate': 0.1, 'subsample': 0.8} ---
[0]	validation_0-mlogloss:3.08677
[50]	validation_0-mlogloss:1.78692
[100]	validation_0-mlogloss:1.56724
[149]	validation_0-mlog

In [8]:
# GRILLE XGBOOST version hardcore pour voir toutes les subtilités histoire de 
# On profite de ma RAM  128g et du GPU pour tester des configs complexes
# Objectif : Battre le score de 0.71

print("\n🚀 LANCEMENT XGBOOST 'EXTREME'...")

# On définit des configurations manuelles spécifiques (plutôt qu'une grille aveugle)
# pour cibler les hyperparamètres qui jouent sur la qualité.

configs_heavy = [
    # Config 1 : Profondeur + Echantillonnage de colonnes (Souvent gagnant sur images)
    {
        "n_estimators": 300,
        "max_depth": 8,
        "learning_rate": 0.05,
        "colsample_bytree": 0.6,  # Ne regarde que 60% des features par arbre
        "subsample": 0.8,
        "name": "XGB_Depth8_ColSample"
    },
    # Config 2 : Beaucoup d'arbres, apprentissage lent (Precision)
    {
        "n_estimators": 500,
        "max_depth": 6,
        "learning_rate": 0.03,    # Tres lent et minitieux
        "colsample_bytree": 0.7,
        "name": "XGB_500_SlowLearn"
    },
    # Config 3 : La totale 
    # Si GPU sature, XGBoost pass sur ma  RAM système grace à ma config
    {
        "n_estimators": 300,
        "max_depth": 10,          # Tres profond
        "learning_rate": 0.05,
        "min_child_weight": 5,    # Regularisation pour eviter overfitting
        "name": "XGB_Deep10_Heavy"
    }
]

for conf in configs_heavy:
    name = conf.pop("name") # on enleve le nom pour ne pas le passer au modele
    print(f"\n🔥 Test Config : {name}")
    print(f"   Params : {conf}")
    
    t0 = time.time()
    try:
        clf = xgb.XGBClassifier(
            objective='multi:softmax',
            num_class=NUM_CLASSES,
            tree_method='hist',
            device='cuda',        # On tente le GPU
            **conf
        )
        
        # verbose=50 pour voir si ca avance
        clf.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            verbose=50
        )
        
        preds = clf.predict(X_val)
        score = f1_score(y_val_cpu, preds, average='weighted')
        duree = time.time() - t0
        
        print(f"   💎 RESULTAT {name} : F1 = {score:.4f} ({duree:.1f}s)")
        
        ALL_RESULTS.append({
            "Famille": "XGBoost_Extreme",
            "Details": f"{name} | {conf}",
            "F1-Score": score,
            "Temps": duree
        })
        
    except Exception as e:
        print(f"   ⚠️ Memoire GPU saturée pour {name}. Passage en mode CPU (RAM 128Go)...")
        # Fallback sur CPU  le GPU craque 
        try:
            clf = xgb.XGBClassifier(
                objective='multi:softmax',
                num_class=NUM_CLASSES,
                tree_method='hist',
                device='cpu',      # Mode CPU force
                n_jobs=-1,         # Tous les coeurs
                **conf
            )
            clf.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=50)
            preds = clf.predict(X_val)
            score = f1_score(y_val_cpu, preds, average='weighted')
            print(f"   💾 SAUVÉ PAR LE CPU : F1 = {score:.4f}")
            
            ALL_RESULTS.append({
                "Famille": "XGBoost_Extreme_CPU",
                "Details": f"{name}",
                "F1-Score": score,
                "Temps": time.time() - t0
            })
        except Exception as e2:
            print(f"   ❌ Echec total : {e2}")

print("\n✅ Tests Extreme finis.")


🚀 LANCEMENT XGBOOST 'EXTREME'...

🔥 Test Config : XGB_Depth8_ColSample
   Params : {'n_estimators': 300, 'max_depth': 8, 'learning_rate': 0.05, 'colsample_bytree': 0.6, 'subsample': 0.8}
[0]	validation_0-mlogloss:3.14945
[50]	validation_0-mlogloss:1.65409
[100]	validation_0-mlogloss:1.33620
[150]	validation_0-mlogloss:1.18618
[200]	validation_0-mlogloss:1.08973
[250]	validation_0-mlogloss:1.01691
[299]	validation_0-mlogloss:0.96057
   💎 RESULTAT XGB_Depth8_ColSample : F1 = 0.7315 (999.1s)

🔥 Test Config : XGB_500_SlowLearn
   Params : {'n_estimators': 500, 'max_depth': 6, 'learning_rate': 0.03, 'colsample_bytree': 0.7}
[0]	validation_0-mlogloss:3.21919
[50]	validation_0-mlogloss:2.08330
[100]	validation_0-mlogloss:1.75367
[150]	validation_0-mlogloss:1.58181
[200]	validation_0-mlogloss:1.47181
[250]	validation_0-mlogloss:1.39363
[300]	validation_0-mlogloss:1.33262
[350]	validation_0-mlogloss:1.28330
[400]	validation_0-mlogloss:1.24183
[450]	validation_0-mlogloss:1.20595
[499]	validatio

KeyboardInterrupt: 

In [10]:
# -XGBOOST ULTIME (MODE CPU / 128GB RAM) 
# On active un log ultra-détaillé pour voir chaque arbre se construire.

print("\n🚜 LANCEMENT XGBOOST 'HEAVY' SUR CPU (RAM 128GO)...")

# Config LOURDE (celle qui a planté sur GPU)
conf_heavy = {
    "n_estimators": 300,
    "max_depth": 10,           # Tres profond = grosse memoire
    "learning_rate": 0.05,
    "min_child_weight": 5,
    "colsample_bytree": 0.7,   # On ajoute ça aussi pour la robustesse
}

print(f"   Parametres : {conf_heavy}")
print("   ⏳ Initialisation... (peut prendre 1-2 min avant le premier affichage)")

t0 = time.time()

try:
    # Force explicite device='cpu' et n_jobs=-1 (tous les coeurs)
    clf = xgb.XGBClassifier(
        objective='multi:softmax',
        num_class=NUM_CLASSES,
        tree_method='hist',   # Le plus rapide sur CPU
        device='cpu',         # ON FORCE LE CPU
        n_jobs=-1,            # TOUTE LA PUISSANCE DU PROCESSEUR
        **conf_heavy
    )
    
    # verbose=1 : Tu verras une ligne pour CHAQUE arbre (0, 1, 2, ... 299)
    # Impossible de douter si ça avance ou pas !
    clf.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        verbose=1 
    )
    
    preds = clf.predict(X_val)
    score = f1_score(y_val_cpu, preds, average='weighted')
    duree = time.time() - t0
    
    print(f"\n✅ SUCCÈS CPU : F1 = {score:.4f} ({duree:.1f}s)")
    
    # On ajoute au tableau final
    ALL_RESULTS.append({
        "Famille": "XGBoost_Heavy_CPU",
        "Details": "Depth:10 | Est:300 | RAM:128Go",
        "F1-Score": score,
        "Temps": duree
    })

except Exception as e:
    print(f"\n❌ Erreur : {e}")

print("\n🏁 Test terminé. Tu peux maintenant lancer la SYNTHESE.")


🚜 LANCEMENT XGBOOST 'HEAVY' SUR CPU (RAM 128GO)...
   Parametres : {'n_estimators': 300, 'max_depth': 10, 'learning_rate': 0.05, 'min_child_weight': 5, 'colsample_bytree': 0.7}
   ⏳ Initialisation... (peut prendre 1-2 min avant le premier affichage)
[0]	validation_0-mlogloss:3.13287
[1]	validation_0-mlogloss:3.00850
[2]	validation_0-mlogloss:2.90567
[3]	validation_0-mlogloss:2.81775
[4]	validation_0-mlogloss:2.73967
[5]	validation_0-mlogloss:2.66967
[6]	validation_0-mlogloss:2.60604
[7]	validation_0-mlogloss:2.54778
[8]	validation_0-mlogloss:2.49369
[9]	validation_0-mlogloss:2.44368
[10]	validation_0-mlogloss:2.39637
[11]	validation_0-mlogloss:2.35267
[12]	validation_0-mlogloss:2.31092
[13]	validation_0-mlogloss:2.27200
[14]	validation_0-mlogloss:2.23504
[15]	validation_0-mlogloss:2.19975
[16]	validation_0-mlogloss:2.16640
[17]	validation_0-mlogloss:2.13433
[18]	validation_0-mlogloss:2.10364
[19]	validation_0-mlogloss:2.07467
[20]	validation_0-mlogloss:2.04666
[21]	validation_0-mloglo

In [6]:
# MODELES ALTERNATIFS GPU 

# install si besoin
import sys
!{sys.executable} -m pip install lightgbm catboost

import lightgbm as lgb
from catboost import CatBoostClassifier

print("\n🚀 BONUS : Test de nouveaux champions GPU...")

# 1. LightGBM (GPU)
print("\n💡 Test LightGBM (GPU)...")
try:
    t0 = time.time()
    lgbm = lgb.LGBMClassifier(
        device='gpu',           # acceleration gpu
        n_estimators=150,
        learning_rate=0.1,
        num_leaves=31,
        verbose=-1              
    )
    lgbm.fit(X_train, y_train)
    p_lgb = lgbm.predict(X_val)
    
    score_lgb = f1_score(y_val_cpu, p_lgb, average='weighted')
    duree = time.time() - t0
    
    print(f"   -> F1: {score_lgb:.4f} ({duree:.1f}s)")
    ALL_RESULTS.append({
        "Famille": "LightGBM",
        "Details": "GPU | N=150 | Leaves=31",
        "F1-Score": score_lgb,
        "Temps": duree
    })
except Exception as e:
    print(f"   ❌ Erreur LightGBM: {e}")

# 2. CatBoost (GPU)
print("\n🐱 Test CatBoost (GPU)...")
try:
    t0 = time.time()
    # Catboost est tres fort par defaut
    cat = CatBoostClassifier(
        task_type="GPU",       # acceleration gpu
        iterations=150,
        learning_rate=0.1,
        depth=6,
        verbose=50             # log tous les 50 arbres
    )
    cat.fit(X_train, y_train)
    p_cat = cat.predict(X_val)
    
    # catboost renvoie parfois des arrays 2D, on aplatit
    p_cat = p_cat.flatten()
    
    score_cat = f1_score(y_val_cpu, p_cat, average='weighted')
    duree = time.time() - t0
    
    print(f"   -> F1: {score_cat:.4f} ({duree:.1f}s)")
    ALL_RESULTS.append({
        "Famille": "CatBoost",
        "Details": "GPU | N=150 | Depth=6",
        "F1-Score": score_cat,
        "Temps": duree
    })
except Exception as e:
    print(f"   ❌ Erreur CatBoost: {e}")

print("\n✅ les modèles complémentaires sont terminés")


🚀 BONUS : Test de nouveaux champions GPU...

💡 Test LightGBM (GPU)...


C:\Users\amisf\anaconda3\envs\masterclass_tooling\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


   -> F1: 0.6958 (393.1s)

🐱 Test CatBoost (GPU)...


0:	learn: 3.1640266	total: 661ms	remaining: 1m 38s
50:	learn: 2.0870322	total: 15s	remaining: 29.1s
100:	learn: 1.8603493	total: 28.7s	remaining: 13.9s
149:	learn: 1.7353702	total: 42.1s	remaining: 0us
   -> F1: 0.4824 (49.5s)

✅ les modèles complémentaires sont terminés


In [5]:
#  MODELES CPU (AVEC LOGS D'ACTIVITE) 
print("\n🌲 lancements modeles cpu...")

# 1. Random Forest (config 1)
print("\n... Random Forest (100 arbres)")
t0 = time.time()
# verbose=3 affiche l'avancement des jobs et des arbres
rf1 = RandomForestClassifier(n_estimators=100, n_jobs=-1, verbose=3)
rf1.fit(X_train, y_train)
p_rf1 = rf1.predict(X_val)

ALL_RESULTS.append({
    "Famille": "RandomForest",
    "Details": "N=100 | Default",
    "F1-Score": f1_score(y_val_cpu, p_rf1, average='weighted'),
    "Temps": time.time() - t0
})

# 2. Random Forest (config 2)
print("\n... Random Forest (200 arbres, entropy)")
t0 = time.time()
# verbose=3 pour voir le defilement
rf2 = RandomForestClassifier(n_estimators=200, criterion='entropy', n_jobs=-1, verbose=3)
rf2.fit(X_train, y_train)
p_rf2 = rf2.predict(X_val)

ALL_RESULTS.append({
    "Famille": "RandomForest",
    "Details": "N=200 | Entropy",
    "F1-Score": f1_score(y_val_cpu, p_rf2, average='weighted'),
    "Temps": time.time() - t0
})

# 3. Regression Logistique
print("\n... Regression Logistique")
t0 = time.time()
# verbose=1 affiche les iterations du solveur
lr = LogisticRegression(max_iter=1000, n_jobs=-1, verbose=1)
lr.fit(X_train, y_train)
p_lr = lr.predict(X_val)

ALL_RESULTS.append({
    "Famille": "LogisticReg",
    "Details": "Baseline",
    "F1-Score": f1_score(y_val_cpu, p_lr, average='weighted'),
    "Temps": time.time() - t0
})


🌲 lancements modeles cpu...

... Random Forest (100 arbres)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.


building tree 14 of 100
building tree 1 of 100
building tree 15 of 100
building tree 16 of 100
building tree 10 of 100
building tree 12 of 100
building tree 13 of 100
building tree 6 of 100
building tree 8 of 100
building tree 3 of 100
building tree 2 of 100
building tree 11 of 100
building tree 4 of 100
building tree 5 of 100
building tree 7 of 100
building tree 9 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
b

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  5.1min finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done 100 out of 100 | elapsed:    1.4s finished



... Random Forest (200 arbres, entropy)


...

[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed: 15.4min


building tree 113 of 200
building tree 114 of 200
building tree 115 of 200
building tree 116 of 200
building tree 117 of 200
building tree 118 of 200
building tree 119 of 200
building tree 120 of 200
building tree 121 of 200
building tree 122 of 200
building tree 123 of 200
building tree 124 of 200
building tree 125 of 200
building tree 126 of 200
building tree 127 of 200
building tree 128 of 200
building tree 129 of 200
building tree 130 of 200
building tree 131 of 200
building tree 132 of 200
building tree 133 of 200
building tree 134 of 200
building tree 135 of 200
building tree 136 of 200
building tree 137 of 200
building tree 138 of 200
building tree 139 of 200
building tree 140 of 200
building tree 141 of 200
building tree 142 of 200
building tree 143 of 200
building tree 144 of 200
building tree 145 of 200
building tree 146 of 200
building tree 147 of 200
building tree 148 of 200
building tree 149 of 200
building tree 150 of 200
building tree 151 of 200
building tree 152 of 200


[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed: 31.2min finished
[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done  96 tasks      | elapsed:    1.3s
[Parallel(n_jobs=16)]: Done 200 out of 200 | elapsed:    2.7s finished
C:\Users\amisf\anaconda3\envs\masterclass_tooling\Lib\site-packages\sklearn\linear_model\_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)



... Regression Logistique


C:\Users\amisf\anaconda3\envs\masterclass_tooling\Lib\site-packages\sklearn\linear_model\_logistic.py:406: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [13]:
# CELLULE A : CLASSEMENT MACHINE LEARNING 
print("\n📊 PODIUM : MACHINE LEARNING CLASSIQUE")

df_all = pd.DataFrame(ALL_RESULTS)

# CORRECTION DU FILTRE : On exclut aussi "DeepLearning"
mask_ml = ~df_all['Famille'].astype(str).str.contains("PyTorch|DL|DeepLearning", case=False, regex=True)
df_ml = df_all[mask_ml].copy()

if not df_ml.empty:
    df_ml = df_ml.sort_values(by="F1-Score", ascending=False).reset_index(drop=True)
    df_ml.index += 1
    df_ml.index.name = 'Rang ML'
    
    pd.set_option('display.max_colwidth', 100)
    print(df_ml[['Famille', 'Details', 'F1-Score', 'Temps']])
    
    df_ml.to_csv(os.path.join(OUTPUT_DIR, "classement_ml_classique.csv"), index=True)
else:
    print("⚠️ Aucun résultat ML classique trouvé.")


📊 PODIUM : MACHINE LEARNING CLASSIQUE
                   Famille  \
Rang ML                      
1        XGBoost_Heavy_CPU   
2          XGBoost_Extreme   
3             RandomForest   
4                  XGBoost   
5             RandomForest   
6                 LightGBM   
7                  XGBoost   
8          XGBoost_Extreme   
9                  XGBoost   
10             LogisticReg   
11                 XGBoost   
12                 XGBoost   
13                 XGBoost   
14                 XGBoost   
15                 XGBoost   
16                CatBoost   

                                                                                                     Details  \
Rang ML                                                                                                        
1                                                                             Depth:10 | Est:300 | RAM:128Go   
2        XGB_Depth8_ColSample | {'n_estimators': 300, 'max_depth': 8, 'learning_rate

In [17]:
print("on passe au mode deep learning pour essayer ")

on passe au mode deep learning pour essayer 


In [25]:
# "Les scores sont bons, mais peut-on aller plus loin avec les features complexes du ResNet ?" , C'est le moment d'importer les librairies de Deep Learning.

print("🧠 Importation des modules de Deep Learning (PyTorch)...")

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

print("🔄 Passage des données en VRAM (Mémoire Vidéo) pour accélération maximale...")

# Conversion explicite Numpy -> PyTorch Tensors
X_train_t = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_t = torch.tensor(y_train, dtype=torch.long).to(device)
X_val_t = torch.tensor(X_val, dtype=torch.float32).to(device)

# Création du DataLoader (Batch Size 4096 car tu as 12Go de VRAM, on en profite)
train_ds = TensorDataset(X_train_t, y_train_t)
gpu_loader = DataLoader(train_ds, batch_size=4096, shuffle=True)

print(f"✅ Architecture Deep Learning prête sur {device_name}.")

🧠 Importation des modules de Deep Learning (PyTorch)...
🔄 Passage des données en VRAM (Mémoire Vidéo) pour accélération maximale...
✅ Architecture Deep Learning prête sur NVIDIA GeForce RTX 4070.


In [1]:
import sys
# désinstalle la version cpu et force la version compatible CUDA (GPU NVIDIA)
!{sys.executable} -m pip uninstall -y torch torchvision torchaudio
!{sys.executable} -m pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Found existing installation: torch 2.7.1+cu118
Uninstalling torch-2.7.1+cu118:
  Successfully uninstalled torch-2.7.1+cu118
Found existing installation: torchvision 0.22.1+cu118
Uninstalling torchvision-0.22.1+cu118:
  Successfully uninstalled torchvision-0.22.1+cu118
Found existing installation: torchaudio 2.7.1+cu118
Uninstalling torchaudio-2.7.1+cu118:
  Successfully uninstalled torchaudio-2.7.1+cu118
Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torch-2.7.1%2Bcu118-cp313-cp313-win_amd64.whl.metadata (27 kB)
  Using cached https://download.pytorch.org/whl/cu118/torchvision-0.22.1%2Bcu118-cp313-cp313-win_amd64.whl.metadata (6.3 kB)
  Using cached https://download.pytorch.org/whl/cu118/torchaudio-2.7.1%2Bcu118-cp313-cp313-win_amd64.whl.metadata (6.8 kB)
Using cached https://download.pytorch.org/whl/cu118/torch-2.7.1%2Bcu118-cp313-cp313-win_amd64.whl (2817.2 MB)
Using cached https://download.pytorch.org/whl/cu118/torchv

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [26]:
#  ACTIVATION MODE DEEP LEARNING (VRAM) 
print("🔄 Transition vers le Deep Learning...")
print("🚀 Envoi des données vers la mémoire GPU (VRAM)...")

# C'est ici qu'on convertit nos tableaux Numpy en Tenseurs PyTorch
# Et qu'on les déplace physiquement sur la carte graphique
X_train_t = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_t = torch.tensor(y_train, dtype=torch.long).to(device)
X_val_t = torch.tensor(X_val, dtype=torch.float32).to(device)

# Création du DataLoader PyTorch (Spécifique au Deep Learning)
# On met un gros batch_size car le GPU aime ça
train_ds = TensorDataset(X_train_t, y_train_t)
gpu_loader = DataLoader(train_ds, batch_size=4096, shuffle=True)

print("✅ Tenseurs PyTorch prêts dans la VRAM. Le réseau de neurones peut démarrer.")

🔄 Transition vers le Deep Learning...
🚀 Envoi des données vers la mémoire GPU (VRAM)...
✅ Tenseurs PyTorch prêts dans la VRAM. Le réseau de neurones peut démarrer.


In [4]:
#  CLASSE PYTORCH DYNAMIQUE 
class ModularMLP(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_layers, dropout_rate, activation_fn):
        super().__init__()
        layers = []
        in_d = input_dim
        
        # choix activation
        if activation_fn == 'relu': act = nn.ReLU()
        elif activation_fn == 'gelu': act = nn.GELU()
        elif activation_fn == 'leaky': act = nn.LeakyReLU(0.1)
        
        # construction couches
        for h_dim in hidden_layers:
            layers.append(nn.Linear(in_d, h_dim))
            layers.append(nn.BatchNorm1d(h_dim))
            layers.append(act)
            layers.append(nn.Dropout(dropout_rate))
            in_d = h_dim
            
        layers.append(nn.Linear(in_d, output_dim))
        self.net = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.net(x)

# FONCTION TEST UNITAIRE 
def run_pytorch_test(config):
    # init model
    model = ModularMLP(
        INPUT_DIM, NUM_CLASSES, 
        config['layers'], config['drop'], config['act']
    ).to(device)
    
    # optimizer
    if config['opt'] == 'adam': optimizer = optim.Adam(model.parameters(), lr=config['lr'])
    elif config['opt'] == 'sgd': optimizer = optim.SGD(model.parameters(), lr=config['lr'], momentum=0.9)
    elif config['opt'] == 'rmsprop': optimizer = optim.RMSprop(model.parameters(), lr=config['lr'])
    
    crit = nn.CrossEntropyLoss()
    
    # train
    model.train()
    t0 = time.time()
    for ep in range(config['epochs']):
        for bx, by in gpu_loader:
            optimizer.zero_grad()
            out = model(bx)
            loss = crit(out, by)
            loss.backward()
            optimizer.step()
            
    duree = time.time() - t0
    
    # eval
    model.eval()
    with torch.no_grad():
        out_val = model(X_val_t)
        _, preds = torch.max(out_val, 1)
    
    preds_cpu = preds.cpu().numpy()
    score = f1_score(y_val_cpu, preds_cpu, average='weighted')
    
    return {
        "Famille": "DeepLearning",
        "Details": f"L:{config['layers']} | Opt:{config['opt']} | Act:{config['act']} | Drop:{config['drop']}",
        "F1-Score": score,
        "Temps": duree
    }

#  GRILLE HYPERPARAMETRES PYTORCH 
# ca va faire beaucoup de modeles !
grid_pt = {
    'layers': [[1024, 512], [2048, 1024, 512], [512, 256]],
    'act': ['relu', 'gelu'],
    'drop': [0.2, 0.5],
    'opt': ['adam', 'rmsprop'],
    'lr': [0.001],
    'epochs': [20] # on fixe a 20 pr aller vite
}

# generation des combinaisons
keys, values = zip(*grid_pt.items())
pt_configs = [dict(zip(keys, v)) for v in itertools.product(*values)]

print(f"🔥 {len(pt_configs)} configurations PyTorch a tester...")

# execution boucle
for i, conf in enumerate(pt_configs):
    print(f"   ... test {i+1}/{len(pt_configs)}: {conf['layers']} - {conf['opt']}")
    res = run_pytorch_test(conf)
    ALL_RESULTS.append(res)

🔥 24 configurations PyTorch a tester...
   ... test 1/24: [1024, 512] - adam
   ... test 2/24: [1024, 512] - rmsprop
   ... test 3/24: [1024, 512] - adam
   ... test 4/24: [1024, 512] - rmsprop
   ... test 5/24: [1024, 512] - adam
   ... test 6/24: [1024, 512] - rmsprop
   ... test 7/24: [1024, 512] - adam
   ... test 8/24: [1024, 512] - rmsprop
   ... test 9/24: [2048, 1024, 512] - adam
   ... test 10/24: [2048, 1024, 512] - rmsprop
   ... test 11/24: [2048, 1024, 512] - adam
   ... test 12/24: [2048, 1024, 512] - rmsprop
   ... test 13/24: [2048, 1024, 512] - adam
   ... test 14/24: [2048, 1024, 512] - rmsprop
   ... test 15/24: [2048, 1024, 512] - adam
   ... test 16/24: [2048, 1024, 512] - rmsprop
   ... test 17/24: [512, 256] - adam
   ... test 18/24: [512, 256] - rmsprop
   ... test 19/24: [512, 256] - adam
   ... test 20/24: [512, 256] - rmsprop
   ... test 21/24: [512, 256] - adam
   ... test 22/24: [512, 256] - rmsprop
   ... test 23/24: [512, 256] - adam
   ... test 24/24: [5

In [20]:
# CLASSEMENT DEEP LEARNING (PYTORCH) 
print("\n🧠 PODIUM : DEEP LEARNING (NEURAL NETWORKS) ")

df_all = pd.DataFrame(ALL_RESULTS)

# CORRECTION DU FILTRE : On garde ce qui contient "DeepLearning" ou "PyTorch"
mask_dl = df_all['Famille'].astype(str).str.contains("PyTorch|DL|DeepLearning", case=False, regex=True)
df_dl = df_all[mask_dl].copy()

if not df_dl.empty:
    df_dl = df_dl.sort_values(by="F1-Score", ascending=False).reset_index(drop=True)
    df_dl.index += 1
    df_dl.index.name = 'Rang DL'
    
    print(df_dl[['Famille', 'Details', 'F1-Score', 'Temps']])
    df_dl.to_csv(os.path.join(OUTPUT_DIR, "classement_deep_learning.csv"), index=True)
else:
    print("⚠️ Aucun résultat Deep Learning trouvé.")


🧠 PODIUM : DEEP LEARNING (NEURAL NETWORKS) 
              Famille  \
Rang DL                 
1        DeepLearning   
2        DeepLearning   
3        DeepLearning   
4        DeepLearning   
5        DeepLearning   
6        DeepLearning   
7        DeepLearning   
8        DeepLearning   
9        DeepLearning   
10       DeepLearning   
11       DeepLearning   
12       DeepLearning   
13       DeepLearning   
14       DeepLearning   
15       DeepLearning   
16       DeepLearning   
17       DeepLearning   
18       DeepLearning   
19       DeepLearning   
20       DeepLearning   
21       DeepLearning   
22       DeepLearning   
23       DeepLearning   
24       DeepLearning   

                                                         Details  F1-Score  \
Rang DL                                                                      
1           L:[2048, 1024, 512] | Opt:adam | Act:gelu | Drop:0.2  0.914124   
2           L:[2048, 1024, 512] | Opt:adam | Act:relu | Drop:0.2  0.90

In [18]:
print("maintenant on vois tout le monde pour faire une comparaison")

maintenant on vois tout le monde pour faire une comparaison


In [19]:
# CLASSEMENT GENERAL FINAL (TOUT CONFONDU) 
print("\n🏆  GRAND VAINQUEUR TOUTES CATEGORIES ")

if ALL_RESULTS:
    df_final = pd.DataFrame(ALL_RESULTS)
    df_final = df_final.sort_values(by="F1-Score", ascending=False).reset_index(drop=True)
    df_final.index += 1
    df_final.index.name = 'Rang Global'
    
    print(df_final[['Famille', 'Details', 'F1-Score', 'Temps']])
    
    path_final = os.path.join(OUTPUT_DIR, "benchmark_complet_final.csv")
    df_final.to_csv(path_final, index=True)
    
    best = df_final.iloc[0]
    print(f"\n🥇 LE CHAMPION EST : {best['Famille']}")
    print(f"   -> Score F1 : {best['F1-Score']:.4f}")
    print(f"   -> Config   : {best['Details']}")
else:
    print("❌ Liste vide.")


🏆  GRAND VAINQUEUR TOUTES CATEGORIES 
                       Famille  \
Rang Global                      
1                 DeepLearning   
2                 DeepLearning   
3                 DeepLearning   
4                 DeepLearning   
5                 DeepLearning   
6                 DeepLearning   
7                 DeepLearning   
8                 DeepLearning   
9                 DeepLearning   
10                DeepLearning   
11                DeepLearning   
12                DeepLearning   
13                DeepLearning   
14                DeepLearning   
15                DeepLearning   
16                DeepLearning   
17                DeepLearning   
18                DeepLearning   
19                DeepLearning   
20           XGBoost_Heavy_CPU   
21                DeepLearning   
22                DeepLearning   
23                DeepLearning   
24                DeepLearning   
25                DeepLearning   
26             XGBoost_Extreme   
27       